In [ ]:
import math
import os
import sys
from math import log

import fiber
import fiber.utils
import fiberutils
import numpy as np
import pandas as pd
import pyarrow.parquet as pq

# from fiber import fiber
from fiber.cohort import Cohort
from fiber.condition import MRNs

from fiber.condition import (
    Diagnosis,
    Drug,
    Encounter,
    LabValue,
    Measurement,
    MRNs,
    Patient,
    Procedure,
    TobaccoUse,
    VitalSign,
)

from fiber.storage import yaml as fiberyaml
from fiber.utils import Timer

from typing import Optional
import matplotlib.pyplot as plt
import seaborn as sns
import venn
import editdistance as ed
from fiber.plots.distributions import bars
import fiber.config

from fiberutils import cohort_utils
from fiberutils.time_series_utils import ssr_transform

import matplotlib.pyplot as plt
from functools import reduce
import json
import inspect
import json
import sys
from pydoc import locate

In [ ]:
#load file
mrn_table = pq.read_table("Merged_Cases_0_with_0.1_.05_threshold.parquet").to_pandas()
mrn_table = mrn_table.reindex(sorted(mrn_table.columns), axis=1)
mrn = mrn_table[["medical_record_number", "age_in_days"]]

#generic function get cohort from mrns
def df_to_cohort(df):
    p_condition = MRNs(df)
    print(MRNs(df))
    cohort = Cohort(p_condition)
    return cohort

#transform dataframe to cohort
mrn = df_to_cohort(mrn)

In [ ]:
#Functions to get detailed information of the FIBER Conditiond to be able to extract the right data
def rename_columns(df,col_ignore):
    replacements = {}
    for column in df.columns:
        replacements[column] = get(column,col_ignore)
    return replacements

def get(feature,col_ignor):
    #context first empty : 
    context=""
    code=""
    
    # does not support time window information inside feature name yet
    if feature.startswith(col_ignor):
        return feature.replace('_', ' ').replace('.', '|')

    split_name = feature.split('__')
    if split_name[1] in [
        i[0]
        for i in inspect.getmembers(
            sys.modules['fiber.condition'],
            inspect.isclass
        )
    ]:
        aggregation = split_name[0]
        split_name = split_name[1:]
    else:
        aggregation = "time series"
        

    if len(split_name) == 3:
        class_name, context, code = split_name
        condition_class = locate(f'fiber.condition.{class_name}')
        description = get_description(condition_class, code, context)
    else:
        class_name, description = split_name
        
    return f'{class_name} | {code }|{description.capitalize()} ({aggregation}) | {context}' 

def get_description( condition_class, code, context):
    return condition_class(
        code=code,
        context=context
    ).patients_per(
        condition_class.description_column
    )[
        condition_class.description_column.name.lower()
    ].iloc[0]

#columns that should be ignored for the condition creation for the timeseries extraction
col_ignore=('age_in_days', 'gender', 'religion', 'race','medical_record_number','HT','patient_ethnic_group','deceased_indicator','mother_account_number','address_zip','marital_status_code')
col_ignore

In [ ]:
# START

In [ ]:
# DIAGNOSIS

In [ ]:
# extracting conditions in needed FIBER format for the time series model

only_diagnosis = [col for col in mrn_table if col.startswith('Diagnosis')]
only_diagnosis

In [ ]:
only_diagnosis_df = mrn_table[
    [#'Diagnosis__EPIC__FOLLOW UP',
 #'Diagnosis__ICD-10__I10',
 'Diagnosis__ICD-9__244.9',
 'Diagnosis__ICD-9__250.00',
 'Diagnosis__ICD-9__272.0',
 'Diagnosis__ICD-9__272.4',
 'Diagnosis__ICD-9__311',
 #'Diagnosis__ICD-9__401.1',
 #'Diagnosis__ICD-9__401.9',
 'Diagnosis__ICD-9__414.00',
 'Diagnosis__ICD-9__414.01',
 'Diagnosis__ICD-9__493.90',
 'Diagnosis__ICD-9__530.81',
 'Diagnosis__ICD-9__786.05',
 'Diagnosis__ICD-9__786.50',
 'Diagnosis__ICD-9__V14',
 'Diagnosis__ICD-9__V14.0',
 'Diagnosis__ICD-9__V15.0',
 'Diagnosis__IMO__114039',
 'Diagnosis__IMO__134988',
 #'Diagnosis__IMO__154288',
 #'Diagnosis__IMO__158530',
 'Diagnosis__IMO__94485'
 #'Diagnosis__SYSTEM__MSDW_NOT APPLICABLE',
 #'Diagnosis__SYSTEM__MSDW_UNKNOWN'
    ]
]

#only_diagnosis_df

In [ ]:
#creating a dictionary for column split up and saving the outcome in the correct condition format
diagnosis_renamed_column=rename_columns(only_diagnosis_df,col_ignore)

#print the outcome of this and use as condition for time series exraction
for n in diagnosis_renamed_column.items(): 
    parts=n[1].split('|')
    diagnosis_statement='Diagnosis(context=\''+str(parts[3]).strip()+'\',code=\''+str(parts[1]).strip()+'\',description=\''+str(parts[2].replace(' (time series) ','')).strip()+'\',data_columns=[\'medical_record_number\',\'age_in_days\',\'description\', \'context_name\',\'context_diagnosis_code\',\'numeric_value\'])|'
    print(diagnosis_statement)
    

In [ ]:
condition = (Diagnosis(context='ICD-9',code='244.9',description='Unspecified acquired hypothyroidism',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='250.00',description='Diabetes mellitus without mention of complication type ii or unspecified type not stated as uncontrolled',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='272.0',description='Pure hypercholesterolemia',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='272.4',description='Other and unspecified hyperlipidemia',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='311',description='Depressive disorder not elsewhere classified',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='414.00',description='Coronary atherosclerosis of unspecified type of vessel native or graft',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='414.01',description='Coronary atherosclerosis of native coronary artery',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='493.90',description='Asthma unspecified',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='530.81',description='Esophageal reflux',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='786.05',description='Shortness of breath',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='786.50',description='Unspecified chest pain',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='V14',description='Personal history of allergy to medicinal agents',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='V14.0',description='Personal history of allergy to penicillin',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='ICD-9',code='V15.0',description='Personal history of allergy other than to medicinal agents',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='IMO',code='114039',description='Hyperlipidemia',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='IMO',code='134988',description='Gerd (gastroesophageal reflux disease)',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value'])|
Diagnosis(context='IMO',code='94485',description='Diabetes mellitus',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value']))


In [ ]:
diagnosis_time_series= mrn.time_series_for(condition)
diagnosis_time_series

In [ ]:
diagnosis_time_series['description'].unique()

In [ ]:
#save as pickle
#diagnosis_time_series.to_pickle("Fetched_Timeseries/Diagnosis/Diagnosis_time_series_all_new.pkl")


In [ ]:
#DRUGS 

In [ ]:
only_drug = [col for col in mrn_table if col.startswith('Drug')]
only_drug

In [ ]:
drug_df = mrn_table[
['Drug__COMPURECORD__Cefazolin',
 'Drug__COMPURECORD__Fentanyl',
 'Drug__COMPURECORD__Glycopyrrolate',
 'Drug__COMPURECORD__Midazolam',
 'Drug__COMPURECORD__Neostigmine',
 'Drug__COMPURECORD__Ondansetron',
 'Drug__COMPURECORD__Propofol',
 'Drug__COMPURECORD__Vecuronium',
 'Drug__EPIC IMMUNIZATION__51',
 'Drug__EPIC MEDICATION__1451',
 'Drug__EPIC MEDICATION__16330',
 'Drug__EPIC MEDICATION__9554',
 'Drug__TDS ALLERGY__10001-MEDS',
 'Drug__TDS ALLERGY__12336-MEDS',
 'Drug__TDS__01060-ACETAMINOPHEN TABS',
 'Drug__TDS__08510-DOCUSATE SODIUM CAP',
 'Drug__TDS__11430-HEPARIN SODIUM INJ',
 'Drug__TDS__17220-OXYCODONE/ACETAMINOPHEN 5/325',
 'Drug__TDS__20535-SODIUM CHLORIDE 0.9%']
]

In [ ]:
drug_renamed_column=rename_columns(drug_df,col_ignore)

for n in drug_renamed_column.items(): 
    parts=n[1].split('|')
    drug_statement='Drug(context=\''+str(parts[3]).strip()+'\',code=\''+str(parts[1]).strip()+'\',description=\''+str(parts[2].replace(' (time series) ','')).strip()+'\',data_columns=[\'medical_record_number\',\'age_in_days\', \'context_name\',\'MATERIAL_NAME\',\'GENERIC_NAME\',\'BRAND1\',\'numeric_value\'])|'
    print(drug_statement)


In [ ]:
conditions = (Drug(context='COMPURECORD',code='Cefazolin',description='Cefazolin',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Fentanyl',description='Fentanyl',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Glycopyrrolate',description='Glycopyrrolate',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Midazolam',description='Midazolam',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Neostigmine',description='Neostigmine',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Ondansetron',description='Ondansetron',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Propofol',description='Propofol',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='COMPURECORD',code='Vecuronium',description='Vecuronium',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='EPIC IMMUNIZATION',code='51',description='Influenza vaccine',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='EPIC MEDICATION',code='1451',description='Acetaminophen 325 mg tablet',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='EPIC MEDICATION',code='16330',description='Aspirin 81 mg tab',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='EPIC MEDICATION',code='9554',description='Oxycodone-acetaminophen 5 mg-325 mg tablet',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS ALLERGY',code='10001-MEDS',description='Meds',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS ALLERGY',code='12336-MEDS',description='Meds',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS',code='01060-ACETAMINOPHEN TABS',description='Acetaminophen tabs',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS',code='08510-DOCUSATE SODIUM CAP',description='Docusate sodium cap',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS',code='11430-HEPARIN SODIUM INJ',description='Heparin sodium inj',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS',code='17220-OXYCODONE/ACETAMINOPHEN 5/325',description='Oxycodone/acetaminophen 5/325',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value'])|
Drug(context='TDS',code='20535-SODIUM CHLORIDE 0.9%',description='Sodium chloride 0.9%',data_columns=['medical_record_number','age_in_days', 'context_name','MATERIAL_NAME','GENERIC_NAME','BRAND1','numeric_value']))




In [ ]:
drug_time_series= mrn.time_series_for(conditions)

In [ ]:
drug_time_series

In [ ]:
 # unique values and feature names: 
drug_time_series['material_name'].unique()

In [ ]:
drug_time_series.loc[drug_time_series['time_delta_in_days'] < 0]

In [ ]:
#check how many patients actually received drugs
#drug_time_series.loc[drug_time_series['material_name'] == NaN]
grouped = drug_time_series.groupby('medical_record_number').count()
grouped.loc[grouped['material_name'] == 0]

In [ ]:
#drug_time_series.to_pickle('Fetched_Timeseries/Drugs/Drug_time_series_all_new.pkl') 

In [ ]:
# LabValues

In [ ]:
for col in mrn_table:
    print(col)

In [ ]:
only_LabValues = [col for col in mrn_table.columns if 'max__LabValue' in col]
only_LabValues

In [ ]:
labvalue_df = mrn_table[
    ['max__LabValue__ALBUMIN, BLD',
 'max__LabValue__ALK PHOSPHATASE, BLD',
 'max__LabValue__ALT(SGPT)',
 'max__LabValue__AMYLASE, BLD',
 'max__LabValue__APTT',
 'max__LabValue__APTT - SL',
 'max__LabValue__AST (SGOT)',
 'max__LabValue__ATRIAL RATE',
 'max__LabValue__BASOPHIL #',
 'max__LabValue__BASOPHIL %',
 'max__LabValue__BILIRUBIN DIRECT',
 'max__LabValue__BILIRUBIN TOTAL',
 'max__LabValue__C REACTIVE PROTEIN HS',
 'max__LabValue__CALCIUM, BLD',
 'max__LabValue__CARBON DIOXIDE-BLD',
 'max__LabValue__CHLORIDE-BLD',
 'max__LabValue__CHOL/HDL CHOL RATIO',
 'max__LabValue__CHOLESTEROL',
 'max__LabValue__CK-MB',
 'max__LabValue__CPK-BLD',
 'max__LabValue__CREATININE-SERUM',
 'max__LabValue__EGFR AFRICAN AM',
 'max__LabValue__EGFR NON-AFRICAN AM',
 'max__LabValue__EJECTION FRACTION',
 'max__LabValue__EOSINOPHIL #',
 'max__LabValue__EOSINOPHIL %',
 'max__LabValue__ERYTHROCYTE SED. RATE',
 'max__LabValue__FERRITIN',
 'max__LabValue__GAMMA GTP-BLD',
 'max__LabValue__GLUCOSE',
 'max__LabValue__HDL CHOLESTEROL',
 'max__LabValue__HEMATOCRIT',
 'max__LabValue__HEMATOCRIT , VEN',
 'max__LabValue__HEMATOCRIT-VEN(POCT)',
 'max__LabValue__HEMOGLOBIN',
 'max__LabValue__HEMOGLOBIN A1C',
 'max__LabValue__INR',
 'max__LabValue__INR - SL',
 'max__LabValue__IRON, SERUM',
 'max__LabValue__LDH-BLD',
 'max__LabValue__LDL CHOLESTEROL',
 'max__LabValue__LDL/HDL RATIO',
 'max__LabValue__LIPASE',
 'max__LabValue__LYMPHOCYTE #',
 'max__LabValue__LYMPHOCYTE %',
 'max__LabValue__MAGNESIUM-BLD',
 'max__LabValue__MEAN CORP. HGB',
 'max__LabValue__MEAN CORP. HGB CONC.',
 'max__LabValue__MEAN CORP. VOLUME',
 'max__LabValue__MEAN PLT VOLUME',
 'max__LabValue__MONOCYTE #',
 'max__LabValue__MONOCYTE %',
 'max__LabValue__NEUTROPHIL #',
 'max__LabValue__NEUTROPHIL %',
 'max__LabValue__NRBC#',
 'max__LabValue__NUCLEATE RBC%',
 'max__LabValue__P AXIS',
 'max__LabValue__P-R INTERVAL',
 'max__LabValue__PATIENT TEMP(POCT)',
 'max__LabValue__PHOSPHORUS-BLD',
 'max__LabValue__PLATELET',
 'max__LabValue__POTASSIUMBLD',
 'max__LabValue__PRO TIME',
 'max__LabValue__PROTEIN TOTAL-BLD',
 'max__LabValue__PROTHROMBIN TIME - SL',
 'max__LabValue__QRS DURATION',
 'max__LabValue__QT',
 'max__LabValue__QTC',
 'max__LabValue__R AXIS',
 'max__LabValue__RBC BLOOD CELL',
 'max__LabValue__RED DISTRIB. WIDTH',
 'max__LabValue__SODIUM-BLD',
 'max__LabValue__T AXIS',
 'max__LabValue__TIBC',
 'max__LabValue__TOT CO2 , VEN(POCT)',
 'max__LabValue__TRANSFERRIN SAT',
 'max__LabValue__TRIGLYCERIDES',
 'max__LabValue__TROPONIN-I',
 'max__LabValue__TSH',
 'max__LabValue__U-BACTERIA',
 'max__LabValue__U-PH',
 'max__LabValue__U-SPECIFIC GRAVITY',
 'max__LabValue__UREA NITROGEN-BLD',
 'max__LabValue__URIC ACID-BLD',
 'max__LabValue__UROBILINOGEN',
 'max__LabValue__VENTRICULAR RATE',
 'max__LabValue__WB CHLORIDE, VEN',
 'max__LabValue__WB CO2, VEN',
 'max__LabValue__WB CREATININE, VEN',
 'max__LabValue__WB GLUCOSE , VEN',
 'max__LabValue__WB GLUCOSE-VEN (POCT)',
 'max__LabValue__WB K , VEN (POCT)',
 'max__LabValue__WB NA , VEN (POCT)',
 'max__LabValue__WB POTASSIUM , VEN',
 'max__LabValue__WB SODIUM , VEN',
 'max__LabValue__WB UREA NITROGEN, VEN',
 'max__LabValue__WHITE BLOOD CELL'] 
]

In [ ]:
labvalue_renamed_column=rename_columns(labvalue_df,col_ignore)

for n in labvalue_renamed_column.items(): 
    parts=n[1].split('|')
    lab_statement='LabValue(\''+str(parts[2].replace(' (max) ','')).strip()+'\')|'
    print(lab_statement)


In [ ]:
conditions= (LabValue('Albumin, bld')|
LabValue('Alk phosphatase, bld')|
LabValue('Alt(sgpt)')|
LabValue('Amylase, bld')|
LabValue('Aptt')|
LabValue('Aptt - sl')|
LabValue('Ast (sgot)')|
LabValue('Atrial rate')|
LabValue('Basophil #')|
LabValue('Basophil %')|
LabValue('Bilirubin direct')|
LabValue('Bilirubin total')|
LabValue('C reactive protein hs')|
LabValue('Calcium, bld')|
LabValue('Carbon dioxide-bld')|
LabValue('Chloride-bld')|
LabValue('Chol/hdl chol ratio')|
LabValue('Cholesterol')|
LabValue('Ck-mb')|
LabValue('Cpk-bld')|
LabValue('Creatinine-serum')|
LabValue('Egfr african am')|
LabValue('Egfr non-african am')|
LabValue('Ejection fraction')|
LabValue('Eosinophil #')|
LabValue('Eosinophil %')|
LabValue('Erythrocyte sed. rate')|
LabValue('Ferritin')|
LabValue('Gamma gtp-bld')|
LabValue('Glucose')|
LabValue('Hdl cholesterol')|
LabValue('Hematocrit')|
LabValue('Hematocrit , ven')|
LabValue('Hematocrit-ven(poct)')|
LabValue('Hemoglobin')|
LabValue('Hemoglobin a1c')|
LabValue('Inr')|
LabValue('Inr - sl')|
LabValue('Iron, serum')|
LabValue('Ldh-bld')|
LabValue('Ldl cholesterol')|
LabValue('Ldl/hdl ratio')|
LabValue('Lipase')|
LabValue('Lymphocyte #')|
LabValue('Lymphocyte %')|
LabValue('Magnesium-bld')|
LabValue('Mean corp. hgb')|
LabValue('Mean corp. hgb conc.')|
LabValue('Mean corp. volume')|
LabValue('Mean plt volume')|
LabValue('Monocyte #')|
LabValue('Monocyte %')|
LabValue('Neutrophil #')|
LabValue('Neutrophil %')|
LabValue('Nrbc#')|
LabValue('Nucleate rbc%')|
LabValue('P axis')|
LabValue('P-r interval')|
LabValue('Patient temp(poct)')|
LabValue('Phosphorus-bld')|
LabValue('Platelet')|
LabValue('Potassiumbld')|
LabValue('Pro time')|
LabValue('Protein total-bld')|
LabValue('Prothrombin time - sl')|
LabValue('Qrs duration')|
LabValue('Qt')|
LabValue('Qtc')|
LabValue('R axis')|
LabValue('Rbc blood cell')|
LabValue('Red distrib. width')|
LabValue('Sodium-bld')|
LabValue('T axis')|
LabValue('Tibc')|
LabValue('Tot co2 , ven(poct)')|
LabValue('Transferrin sat')|
LabValue('Triglycerides')|
LabValue('Troponin-i')|
LabValue('Tsh')|
LabValue('U-bacteria')|
LabValue('U-ph')|
LabValue('U-specific gravity')|
LabValue('Urea nitrogen-bld')|
LabValue('Uric acid-bld')|
LabValue('Urobilinogen')|
LabValue('Ventricular rate')|
LabValue('Wb chloride, ven')|
LabValue('Wb co2, ven')|
LabValue('Wb creatinine, ven')|
LabValue('Wb glucose , ven')|
LabValue('Wb glucose-ven (poct)')|
LabValue('Wb k , ven (poct)')|
LabValue('Wb na , ven (poct)')|
LabValue('Wb potassium , ven')|
LabValue('Wb sodium , ven')|
LabValue('Wb urea nitrogen, ven')|
LabValue('White blood cell'))

In [ ]:
labvalue_time_series= mrn.time_series_for(conditions)

In [ ]:
labvalue_time_series

In [ ]:
#labvalue_time_series.to_pickle('Fetched_Timeseries/LabValues/labvalue_time_series_all.pkl') 

In [ ]:
for n in labvalue_time_series['test_name'].unique(): 
    print(n)

In [ ]:
labvalue_time_series.loc[labvalue_time_series['time_delta_in_days'] < 0]

In [ ]:
#check how many patients actually received drugs
grouped = labvalue_time_series.groupby('medical_record_number').count()
grouped.loc[grouped['test_name'] == 0]

In [ ]:
#VitalSign


In [ ]:
only_VitalSign = [col for col in mrn_table.columns if 'max__VitalSign__TDS' in col]
only_VitalSign

In [ ]:
vitalsign_df = mrn_table[['max__VitalSign__EPIC__(RESP)',
 'max__VitalSign__EPIC__(T-O)',
 'max__VitalSign__EPIC__(T-T)',
 'max__VitalSign__EPIC__DBP',
 'max__VitalSign__EPIC__PAIN SCORE',
 'max__VitalSign__EPIC__PULSE',
 'max__VitalSign__EPIC__PULSE OXIMETRY',
 'max__VitalSign__EPIC__SBP',
 'max__VitalSign__EPIC__TEMPERATURE',
 'max__VitalSign__IBEX__(RESP)',
 'max__VitalSign__IBEX__(T-O)',
 'max__VitalSign__IBEX__(T-T)',
 'max__VitalSign__IBEX__DBP',
 'max__VitalSign__IBEX__PAIN SCORE',
 'max__VitalSign__IBEX__PULSE',
 'max__VitalSign__IBEX__PULSE OXIMETRY',
 'max__VitalSign__IBEX__SBP',
 'max__VitalSign__IBEX__TEMPERATURE',
 'max__VitalSign__TDS__(P-R)',
 'max__VitalSign__TDS__(RESP)',
 'max__VitalSign__TDS__(T-O)',
 'max__VitalSign__TDS__(T-T)',
 'max__VitalSign__TDS__ADMIT WT',
 'max__VitalSign__TDS__APS PAIN NOW',
 'max__VitalSign__TDS__APS WORST EXPERIENCED',
 'max__VitalSign__TDS__DBP',
 'max__VitalSign__TDS__PAIN SCORE',
 'max__VitalSign__TDS__SBP']]


In [ ]:
#only BP

vitalsign_df = mrn_table[[
 'max__VitalSign__EPIC__DBP',
 'max__VitalSign__EPIC__SBP',
 'max__VitalSign__IBEX__DBP',
 'max__VitalSign__IBEX__SBP',
 'max__VitalSign__TDS__DBP',
 'max__VitalSign__TDS__SBP']]


In [ ]:
vitalsign_renamed_column=rename_columns(vitalsign_df,col_ignore)

for n in vitalsign_renamed_column.items(): 
    parts=n[1].split('|')
    vital_statement='VitalSign(context=\''+str(parts[3]).strip()+'\',code=\''+str(parts[1]).strip()+'\',description=\''+str(parts[2].replace(' (max) ','')).strip()+'\')|'
    print(vital_statement)


In [ ]:
#only BP

conditions = (VitalSign(context='EPIC',code='DBP',description='Diastolic blood pressure')|
VitalSign(context='EPIC',code='SBP',description='Systolic blood pressure')|
VitalSign(context='IBEX',code='DBP',description='Diastolic blood pressure')|
VitalSign(context='IBEX',code='SBP',description='Systolic blood pressure')|
VitalSign(context='TDS',code='DBP',description='Diastolic blood pressure')|
VitalSign(context='TDS',code='SBP',description='Systolic blood pressure'))


In [ ]:
conditions = (VitalSign(context='EPIC',code='(RESP)',description='Respirations')|
VitalSign(context='EPIC',code='(T-O)',description='Temperature, oral')|
VitalSign(context='EPIC',code='(T-T)',description='Temperature, tympanic')|
VitalSign(context='EPIC',code='DBP',description='Diastolic blood pressure')|
VitalSign(context='EPIC',code='PAIN SCORE',description='Pain score')|
VitalSign(context='EPIC',code='PULSE',description='Pulse')|
VitalSign(context='EPIC',code='PULSE OXIMETRY',description='O2 saturation')|
VitalSign(context='EPIC',code='SBP',description='Systolic blood pressure')|
VitalSign(context='EPIC',code='TEMPERATURE',description='Temperature')|
#VitalSign(context='IBEX',code='(RESP)',description='Respirations')|
#VitalSign(context='IBEX',code='(T-O)',description='Temperature, oral')|
#VitalSign(context='IBEX',code='(T-T)',description='Temperature, tympanic')|
#VitalSign(context='IBEX',code='DBP',description='Diastolic blood pressure')|
#VitalSign(context='IBEX',code='PAIN SCORE',description='Pain score')|
#VitalSign(context='IBEX',code='PULSE',description='Pulse')|
#VitalSign(context='IBEX',code='PULSE OXIMETRY',description='O2 saturation')|
#VitalSign(context='IBEX',code='SBP',description='Systolic blood pressure')|
#VitalSign(context='IBEX',code='TEMPERATURE',description='Temperature')|
VitalSign(context='TDS',code='(P-R)',description='Pulse, radial')|
#VitalSign(context='TDS',code='(RESP)',description='Respirations')|
#VitalSign(context='TDS',code='(T-O)',description='Temperature, oral')|
#VitalSign(context='TDS',code='(T-T)',description='Temperature, tympanic')|
VitalSign(context='TDS',code='ADMIT WT',description='Admit wt')|
VitalSign(context='TDS',code='APS PAIN NOW',description='Pain now')|
VitalSign(context='TDS',code='APS WORST EXPERIENCED',description='Worst experienced'))
#VitalSign(context='TDS',code='DBP',description='Diastolic blood pressure')|
#VitalSign(context='TDS',code='PAIN SCORE',description='Pain score')|
#VitalSign(context='TDS',code='SBP',description='Systolic blood pressure')


In [ ]:
vitalsign_time_series= mrn.time_series_for(conditions)

In [ ]:
vitalsign_time_series

In [ ]:
vitalsign_time_series.to_pickle('Fetched_Timeseries/VitalSigns/vitalsign_time_series_all_BP.pkl') 

In [ ]:
#save as pickle
#vitalsign_time_series.to_pickle('Fetched_Timeseries/VitalSigns/vitalsign_time_series_all.pkl') 

In [ ]:
for n in vitalsign_time_series['context_procedure_code'].unique(): 
    print(n)

In [ ]:
vitalsign_time_series.loc[vitalsign_time_series['time_delta_in_days'] < 0]

In [ ]:
#check how many patients actually received drugs
grouped = vitalsign_time_series.groupby('medical_record_number').count()
grouped.loc[grouped['context_procedure_code'] == 0]

In [ ]:
# Procedure

In [ ]:
only_Procedure = [col for col in mrn_table.columns if 'Procedure' in col]
only_Procedure

In [ ]:
procedure_df = mrn_table[
    ['Procedure__CAREFUSION LAB__6TEST',
 'Procedure__CAREFUSION LAB__AMY',
 'Procedure__CAREFUSION LAB__APT-S',
 'Procedure__CAREFUSION LAB__APTT',
 'Procedure__CAREFUSION LAB__AST',
 'Procedure__CAREFUSION LAB__BMP',
 'Procedure__CAREFUSION LAB__C&P',
 'Procedure__CAREFUSION LAB__CMP',
 'Procedure__CAREFUSION LAB__CPD',
 'Procedure__CAREFUSION LAB__CREA',
 'Procedure__CAREFUSION LAB__CTP',
 'Procedure__CAREFUSION LAB__EGFRG',
 'Procedure__CAREFUSION LAB__ER-VP',
 'Procedure__CAREFUSION LAB__HFP',
 'Procedure__CAREFUSION LAB__LIPID',
 'Procedure__CAREFUSION LAB__LIV',
 'Procedure__CAREFUSION LAB__MG-H',
 'Procedure__CAREFUSION LAB__P9117',
 'Procedure__CAREFUSION LAB__PM',
 'Procedure__CAREFUSION LAB__PO4',
 'Procedure__CAREFUSION LAB__PRO',
 'Procedure__CAREFUSION LAB__PT',
 'Procedure__CAREFUSION LAB__PT-S',
 'Procedure__CAREFUSION LAB__SLIDE',
 'Procedure__CAREFUSION LAB__TS-G',
 'Procedure__CAREFUSION LAB__TSH',
 'Procedure__CAREFUSION LAB__UCHEM',
 'Procedure__CAREFUSION LAB__UMIC',
 'Procedure__CPT-4__71010',
 'Procedure__CPT-4__71020',
 'Procedure__CPT-4__80048',
 'Procedure__CPT-4__80053',
 'Procedure__CPT-4__80061',
 'Procedure__CPT-4__80076',
 'Procedure__CPT-4__81003',
 'Procedure__CPT-4__81015',
 'Procedure__CPT-4__82565',
 'Procedure__CPT-4__83735',
 'Procedure__CPT-4__84100',
 'Procedure__CPT-4__84443',
 'Procedure__CPT-4__84450',
 'Procedure__CPT-4__85008',
 'Procedure__CPT-4__85025',
 'Procedure__CPT-4__85027',
 'Procedure__CPT-4__85610',
 'Procedure__CPT-4__85730',
 'Procedure__CPT-4__93000',
 'Procedure__EKG__93000-ELECTROCARDIOGRAM, COMPLETE',
 'Procedure__EPIC CPT-4__100001',
 'Procedure__EPIC CPT-4__100070',
 'Procedure__EPIC CPT-4__100084',
 'Procedure__EPIC CPT-4__100107',
 'Procedure__EPIC CPT-4__100269',
 'Procedure__EPIC CPT-4__100314',
 'Procedure__EPIC CPT-4__100516',
 'Procedure__EPIC CPT-4__101071',
 'Procedure__EPIC__(RESP)',
 'Procedure__EPIC__(T-O)',
 'Procedure__EPIC__(T-T)',
 'Procedure__EPIC__114290',
 'Procedure__EPIC__117280',
 'Procedure__EPIC__117417',
 'Procedure__EPIC__119049',
 'Procedure__EPIC__151490',
 'Procedure__EPIC__6029',
 'Procedure__EPIC__6032',
 'Procedure__EPIC__6034',
 'Procedure__EPIC__6037',
 'Procedure__EPIC__6104',
 'Procedure__EPIC__6107',
 'Procedure__EPIC__61970',
 'Procedure__EPIC__62452',
 'Procedure__EPIC__63072',
 'Procedure__EPIC__63108',
 'Procedure__EPIC__63112',
 'Procedure__EPIC__6372',
 'Procedure__EPIC__6432',
 'Procedure__EPIC__64416',
 'Procedure__EPIC__64444',
 'Procedure__EPIC__64490',
 'Procedure__EPIC__64824',
 'Procedure__EPIC__64914',
 'Procedure__EPIC__64924',
 'Procedure__EPIC__6501',
 'Procedure__EPIC__6550',
 'Procedure__EPIC__6551',
 'Procedure__EPIC__6624',
 'Procedure__EPIC__6635',
 'Procedure__EPIC__7509',
 'Procedure__EPIC__BP SITE',
 'Procedure__EPIC__DBP',
 'Procedure__EPIC__HEIGHT',
 'Procedure__EPIC__PAIN SCORE',
 'Procedure__EPIC__PULSE',
 'Procedure__EPIC__PULSE OXIMETRY',
 'Procedure__EPIC__R APACHE TEMPERATURE',
 'Procedure__EPIC__SBP',
 'Procedure__EPIC__TEMPERATURE',
 'Procedure__EPIC__WEIGHT',
 #'Procedure__IBEX__(RESP)',
 #'Procedure__IBEX__(T-O)',
 #'Procedure__IBEX__(T-T)',
# 'Procedure__IBEX__DBP',
# 'Procedure__IBEX__PAIN SCORE',
# 'Procedure__IBEX__PULSE',
# 'Procedure__IBEX__PULSE OXIMETRY',
 #'Procedure__IBEX__SBP',
# 'Procedure__IBEX__TEMPERATURE',
# 'Procedure__IBEX__WEIGHT',
 'Procedure__ICD-9__89.02',
 'Procedure__IDXRAD__CH2PAL-PA,LL, CXR',
 'Procedure__MUSE ID__93000',
 'Procedure__POWERPATH__10430',
 #'Procedure__SYSTEM__MSDW_NOT APPLICABLE',
# 'Procedure__SYSTEM__MSDW_UNKNOWN',
 'Procedure__TAMTRON EAP__100001',
 'Procedure__TDS ORDER__ADMISSION',
 'Procedure__TDS ORDER__CARDIOLOGY',
 'Procedure__TDS ORDER__DIET',
 'Procedure__TDS ORDER__MDNUR',
 'Procedure__TDS__(P-R)',
# 'Procedure__TDS__(RESP)',
# 'Procedure__TDS__(T-O)',
# 'Procedure__TDS__(T-T)',
 'Procedure__TDS__ADMIT WT',
 'Procedure__TDS__APS NOW',
 'Procedure__TDS__APS PAIN LEVEL IS ACCEPTABLE TO PATIENT',
 'Procedure__TDS__APS PAIN NOW',
 'Procedure__TDS__APS WORST EXPERIENCED']]
# 'Procedure__TDS__DBP',
# 'Procedure__TDS__HEIGHT',
# 'Procedure__TDS__PAIN SCORE',
# 'Procedure__TDS__SBP',
# 'Procedure__TDS__WEIGHT'

In [ ]:
procedure_renamed_column=rename_columns(procedure_df,col_ignore)

for n in procedure_renamed_column.items(): 
    parts=n[1].split('|')
    procedures_statement='Procedure(context=\''+str(parts[3]).strip()+'\',code=\''+str(parts[1]).strip()+'\',description=\''+str(parts[2].replace(' (time series) ','')).strip()+'\',data_columns=[\'medical_record_number\',\'age_in_days\', \'context_name\',\'PROCEDURE_DESCRIPTION\',\'numeric_value\'])|'
    print(procedures_statement)


In [ ]:
conditions= (Procedure(context='CAREFUSION LAB',code='6TEST',description='6-test',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='AMY',description='Hemoglobin a1c',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='APT-S',description='Ptt-stat lab',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='APTT',description='Partial thromboplastin time',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='AST',description='Ast (sgot)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='BMP',description='Basic metabolic panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='C&P',description='Cbc+platelet',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='CMP',description='Comprehensive metabolic panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='CPD',description='Cbc+plt+diff',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='CREA',description='Creatinine-serum',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='CTP',description='Cbc+platelet',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='EGFRG',description='Egfr test calculation',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='ER-VP',description='Venous panel (er)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='HFP',description='Hepatic function panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='LIPID',description='Lipid panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='LIV',description='Liver transp monit',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='MG-H',description='Magnesium-blood',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='P9117',description='Chem 7',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='PM',description='Pat. monitoring',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='PO4',description='Phosphorus-blood',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='PRO',description='Prothrombin time',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='PT',description='Prothrombin time',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='PT-S',description='Pt-stat lab',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='SLIDE',description='Bld smear, microscopic',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='TS-G',description='Type and screen',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='TSH',description='Tsh',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='UCHEM',description='Urinalysis, routine',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CAREFUSION LAB',code='UMIC',description='Microscopic exam of urine',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='71010',description='Radiologic examination, chest; single view, frontal',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='71020',description='Radiologic examination, chest, 2 views, frontal and lateral;',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='80048',description='Basic metabolic panel (calcium, total) this panel must include the following: calcium, total (82310), carbon dioxide (82374), chloride (82435), creatinine (82565), glucose (82947), potassium (84132), sodium (84295), urea nitrogen (bun) (84520)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='80053',description='Comprehensive metabolic panel this panel must include the following: albumin (82040), bilirubin, total (82247), calcium, total (82310), carbon dioxide (bicarbonate) (82374), chloride (82435), creatinine (82565), glucose (82947), phosphatase, alkaline (84075), potassium (84132), protein, total (84155), sodium (84295), transferase, alanine amino (alt) (sgpt) (84460), transferase, aspartate amino (ast) (sgot) (84450), urea nitrogen (bun) (84520)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='80061',description='Lipid panel this panel must include the following: cholesterol, serum, total (82465), lipoprotein, direct measurement, high density cholesterol (hdl cholesterol) (83718), triglycerides (84478)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='80076',description='Hepatic function panel this panel must include the following: albumin (82040), bilirubin, total (82247), bilirubin, direct (82248), phosphatase, alkaline (84075), protein, total (84155), transferase, alanine amino (alt) (sgpt) (84460), transferase, aspartate amino (ast) (sgot) (84450)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='81003',description='Urinalysis, by dip stick or tablet reagent for bilirubin, glucose, hemoglobin, ketones, leukocytes, nitrite, ph, protein, specific gravity, urobilinogen, any number of these constituents; automated, without microscopy',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='81015',description='Urinalysis; microscopic only',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='82565',description='Creatinine; blood',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='83735',description='Magnesium',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='84100',description='Phosphorus inorganic (phosphate);',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='84443',description='Thyroid stimulating hormone (tsh)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='84450',description='Transferase; aspartate amino (ast) (sgot)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='85008',description='Blood count; blood smear, microscopic examination without manual differential wbc count',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='85025',description='Blood count; complete (cbc), automated (hgb, hct, rbc, wbc and platelet count) and automated differential wbc count',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='85027',description='Blood count; complete (cbc), automated (hgb, hct, rbc, wbc and platelet count)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='85610',description='Prothrombin time;',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='85730',description='Thromboplastin time, partial (ptt); plasma or whole blood',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='CPT-4',code='93000',description='Electrocardiogram, routine ecg with at least 12 leads; with interpretation and report',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EKG',code='93000-ELECTROCARDIOGRAM, COMPLETE',description='Electrocardiogram, complete',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100001',description='Surgical pathology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100070',description='6-test',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100084',description='Pat. monitoring',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100107',description='Venous panel (er)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100269',description='Egfr test calculation',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100314',description='Tsh',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='100516',description='Type and screen',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC CPT-4',code='101071',description='Cytopathology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='(RESP)',description='Respirations',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='(T-O)',description='Temperature, oral',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='(T-T)',description='Temperature, tympanic',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='114290',description='Tsh',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='117280',description='Liver transp monit',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='117417',description='Chem 7',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='119049',description='Type and screen',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='151490',description='Cytopathology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6029',description='Basic metabolic panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6032',description='Comprehensive metabolic panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6034',description='Lipid panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6037',description='Hepatic function panel',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6104',description='Urinalysis, routine',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6107',description='Microscopic exam of urine',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='61970',description='Surgical pathology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='62452',description='Creatinine-serum',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='63072',description='Bld smear, microscopic',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='63108',description='Pt-stat lab',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='63112',description='Ptt-stat lab',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6372',description='Magnesium-blood',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6432',description='Phosphorus-blood',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='64416',description='6-test',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='64444',description='Pat. monitoring',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='64490',description='Venous panel (er)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='64824',description='Egfr test calculation',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='64914',description='X-ray chest 1 view portable',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='64924',description='X-ray chest pa and lateral only',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6501',description='Ast (sgot)',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6550',description='Cbc+plt+diff',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6551',description='Cbc+platelet',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6624',description='Prothrombin time',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='6635',description='Partial thromboplastin time',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='7509',description='Electrocardiogram, complete',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='BP SITE',description='Bp site',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='DBP',description='Diastolic blood pressure',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='HEIGHT',description='Height',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='PAIN SCORE',description='Pain score',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='PULSE',description='Pulse',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='PULSE OXIMETRY',description='O2 saturation',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='R APACHE TEMPERATURE',description='R apache temperature',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='SBP',description='Systolic blood pressure',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='TEMPERATURE',description='Temperature',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='EPIC',code='WEIGHT',description='Weight',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='ICD-9',code='89.02',description='Limited interview/evalua',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='IDXRAD',code='CH2PAL-PA,LL, CXR',description='Pa,ll, cxr',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='MUSE ID',code='93000',description='Electrocardiogram, complete',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='POWERPATH',code='10430',description='Pathology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TAMTRON EAP',code='100001',description='Surgical pathology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS ORDER',code='ADMISSION',description='Admission',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS ORDER',code='CARDIOLOGY',description='Cardiology',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS ORDER',code='DIET',description='Diet',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS ORDER',code='MDNUR',description='Md to nurse',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS',code='(P-R)',description='Pulse, radial',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS',code='ADMIT WT',description='Admit wt',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS',code='APS NOW',description='Now',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS',code='APS PAIN LEVEL IS ACCEPTABLE TO PATIENT',description='Pain level is acceptable to patient',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS',code='APS PAIN NOW',description='Pain now',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value'])|
Procedure(context='TDS',code='APS WORST EXPERIENCED',description='Worst experienced',data_columns=['medical_record_number','age_in_days', 'context_name','PROCEDURE_DESCRIPTION','numeric_value']))


In [ ]:
procedure_time_series= mrn.time_series_for(conditions)

In [ ]:
procedure_time_series

In [ ]:
#save as pickle
#procedure_time_series.to_pickle('Fetched_Timeseries/Procedure/procedure_time_series_all.pkl')


In [ ]:
for n in procedure_time_series['procedure_description'].unique(): 
    print(n)
    

In [ ]:
procedure_time_series.loc[procedure_time_series['time_delta_in_days'] < 0]


In [ ]:
#check how many patients actually received drugs
grouped = procedure_time_series.groupby('medical_record_number').count()
grouped.loc[grouped['procedure_description'] == 0]

In [ ]:
grouped

In [ ]:
#checking for time delta

#if delta is smaller than 0, the diagnosis has been there before HT_onset
diagnosis_time_series.loc[diagnosis_time_series['time_delta_in_days'] < 0]

#check for entire cohort being there
diagnosis_time_series.groupby('medical_record_number').count()

diagnosis_time_series.loc[diagnosis_time_series['medical_record_number'] == '1000133708']

#checking for time_delta_in_days 
check_df = diagnosis_time_series.loc[diagnosis_time_series['medical_record_number'] == '1000133708']
cohort = df_to_cohort(check_df)
condition=(Diagnosis(context='APRDRG MDC',code='005',description='Circulatory system diseases',data_columns=['medical_record_number','age_in_days','description', 'context_name','context_diagnosis_code','numeric_value']))

#conditions = Diagnosis()
x = cohort.get(condition)
x
